In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import warnings

warnings.filterwarnings('ignore')

In [3]:
sales_data = pd.read_csv(r'C:\Users\amank\OneDrive\Desktop\Car_Forcasting\data\processed_data\sales_data_clean.csv')

In [4]:
sales_data.head()

,STOCKNO,INVOICEDAT,BRANCHNO,Make,Model,SALETYPE,SALEPRICE,COSTPRICE,METALPROFIT,Mileage,Make_new,Model_new
0,5,2021-05-21,1,ford,Ford Transit Variants,New Retail,33165.0,28857.75,4307.250000,NaN,ford,Ford Transit Variants
1,8,2021-06-28,1,ford,Ford Focus Variants,Used Sale BFC VAT Q,13332.5,10833.33,2499.170000,12959.0,ford,Ford Focus Variants
2,10,2021-06-03,1,ford,ecosport,Used Sale BFC Margin,12299.0,11050.00,1040.833333,21837.0,ford,ecosport
3,12,2021-07-07,6,ford,Ford Fiesta Variants,Used Sale BFC Margin,14299.0,13000.00,1082.500000,14842.0,ford,Ford Fiesta Variants
4,16,2021-05-14,1,ford,Ford Focus Variants,Used Sale BFC Margin,13710.0,13150.00,466.666667,16250.0,ford,Ford Focus Variants


In [5]:
sales_data = sales_data[['INVOICEDAT', 'Make_new']]

In [6]:
sales_data

,INVOICEDAT,Make_new
0,2021-05-21,ford
1,2021-06-28,ford
2,2021-06-03,ford
3,2021-07-07,ford
4,2021-05-14,ford
...,...,...
33404,2024-01-31,ford
33405,2024-01-31,ford
33406,2024-01-31,ford
33407,2024-01-12,ford


In [7]:
sales_data.Make_new.value_counts()

Make_new
ford             24894
vauxhall          1028
volkswagen         793
bmw                630
Other              619
mercedes-benz      564
audi               515
nissan             467
peugeot            404
tesla              365
seat               324
renault            271
fiat               264
hyundai            256
citroen            252
toyota             248
mini               242
land rover         220
kia                209
skoda              205
volvo              178
honda              170
mitsubishi         144
jaguar             141
Name: count, dtype: int64

In [8]:
import pandas as pd
from prophet import Prophet
import numpy as np

# Convert your data's date column to datetime
sales_data['INVOICEDAT'] = pd.to_datetime(sales_data['INVOICEDAT'])

# Set the forecast period to 90 days (approximately three months)
forecast_period = 90
monthly_forecasts = {}

# Loop through each unique make
for make in sales_data['Make_new'].unique():
    # Filter data for the current make and aggregate to daily frequency
    make_data = sales_data[sales_data['Make_new'] == make]
    make_daily = make_data.groupby('INVOICEDAT').size().reset_index(name='y')
    make_daily = make_daily.rename(columns={'INVOICEDAT': 'ds'})
    
    # Check if there are at least two valid rows
    if make_daily['y'].count() < 2:
        print(f"Skipping '{make}' due to insufficient data.")
        continue
    
    # Initialize and fit Prophet model
    model = Prophet()
    model.fit(make_daily)
    
    # Generate future dates for 3 months (90 days)
    future = model.make_future_dataframe(periods=forecast_period)
    forecast = model.predict(future)
    
    # Filter forecast for the next three months only
    forecast['month'] = forecast['ds'].dt.to_period('M')  # Extract month from date
    next_three_months = forecast[forecast['ds'] > make_daily['ds'].max()].groupby('month')['yhat'].sum().head(3)
    
    # Store results for each make in dictionary
    monthly_forecasts[make] = next_three_months.values

# Convert the dictionary into a DataFrame with each row as a make and columns as months
forecast_df = pd.DataFrame.from_dict(
    monthly_forecasts, 
    orient='index', 
    columns=['First Month', 'Second Month', 'Third Month']
).reset_index().rename(columns={'index': 'Make'})


19:03:40 - cmdstanpy - INFO - Chain [1] start processing
19:03:40 - cmdstanpy - INFO - Chain [1] done processing
19:03:41 - cmdstanpy - INFO - Chain [1] start processing
19:03:41 - cmdstanpy - INFO - Chain [1] done processing
19:03:42 - cmdstanpy - INFO - Chain [1] start processing
19:03:42 - cmdstanpy - INFO - Chain [1] done processing
19:03:42 - cmdstanpy - INFO - Chain [1] start processing
19:03:42 - cmdstanpy - INFO - Chain [1] done processing
19:03:42 - cmdstanpy - INFO - Chain [1] start processing
19:03:43 - cmdstanpy - INFO - Chain [1] done processing
19:03:43 - cmdstanpy - INFO - Chain [1] start processing
19:03:43 - cmdstanpy - INFO - Chain [1] done processing
19:03:43 - cmdstanpy - INFO - Chain [1] start processing
19:03:44 - cmdstanpy - INFO - Chain [1] done processing
19:03:44 - cmdstanpy - INFO - Chain [1] start processing
19:03:44 - cmdstanpy - INFO - Chain [1] done processing
19:03:44 - cmdstanpy - INFO - Chain [1] start processing
19:03:45 - cmdstanpy - INFO - Chain [1]

Skipping 'nan' due to insufficient data.


19:03:52 - cmdstanpy - INFO - Chain [1] start processing
19:03:52 - cmdstanpy - INFO - Chain [1] done processing


In [9]:
forecast = forecast_df.set_index('Make').astype(int)

In [10]:
forecast

,First Month,Second Month,Third Month
Make,,,
ford,807,1038,811
Other,58,63,53
volvo,41,38,33
fiat,1,38,38
seat,47,46,40
mini,36,43,26
vauxhall,67,58,67
renault,35,34,41
mitsubishi,1,35,40


In [11]:
forecast.sum()

First Month     1590
Second Month    2073
Third Month     1791
dtype: int64